# Database Connection

In [1]:
# Dependencies

import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

You will need to create a `.env` file in your project root directory and add the following environment variables:

DB_USER=your_database_user

DB_PASSWORD=your_database_password

DB_HOST=your_database_host

DB_PORT=your_database_port

DB_NAME=your_database_name

You will also need to add the .env file to your `.gitignore` file to prevent it from being tracked by Git. The .gitignore file should contain the following line:

`.env`

---



In Windows Command Prompt:

C:\Program Files\PostgreSQL\16\bin>psql -U postgres -h localhost -d global_inflation_db
Password for user postgres:

psql (16.3)
WARNING: Console code page (437) differs from Windows code page (1252)
         8-bit characters might not work correctly. See psql reference
         page "Notes for Windows users" for details.
Type "help" for help.

global_inflation_db=#

In [ ]:
# Load environment variables from .env file
load_dotenv()

# Get environment variables
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')

# Print environment variables to verify they are loaded correctly
print("DB_USER:", db_user)
print("DB_PASSWORD:", db_password)
print("DB_HOST:", db_host)
print("DB_PORT:", db_port)
print("DB_NAME:", db_name)


In [3]:
# Function to connect to DB
def connect_to_db():
    try:
        connection = psycopg2.connect(
            user=os.getenv('DB_USER'),
            password=os.getenv('DB_PASSWORD'),
            host=os.getenv('DB_HOST'),
            port=int(os.getenv('DB_PORT')),  # Convert port to integer
            database=os.getenv('DB_NAME')
        )
        return connection
    except Exception as error:
        print(f"Error: {error}")
        return None

In [4]:
# Create the database connection string
connection_string = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'

# Create a SQLAlchemy engine
engine = create_engine(connection_string)

In [5]:
# Save to .csv file
# Directory to save the extracted CSV files
output_directory = "C:\\Users\\wware\\Desktop\\UWA Bootcamp\\Challenges\\Global_Inflation_Trends_Dashboard\\1_integrated_code\\DB_Extraction"


In [6]:
# Function to fetch data, sort, and save to CSV
def fetch_data_and_save_to_csv(table_name):
    query = f"SELECT * FROM {table_name}"
    try:
        df = pd.read_sql_query(query, engine)
        if df is not None:
            print(f"\nData fetched successfully from {table_name}:")
            print(df.head())  # Print the first few rows of the DataFrame to verify
            
            # Sort the DataFrame by 'country' and then 'year' if the columns exist
            if 'country' in df.columns and 'year' in df.columns:
                df = df.sort_values(by=['country', 'year'])
            elif 'country' in df.columns:
                df = df.sort_values(by='country')
            elif 'year' in df.columns:
                df = df.sort_values(by='year')

            # High-level assessment
            print(f"\nHigh-level Assessment of {table_name} Data:")
            print(f"Number of records: {len(df)}")
            
            # Number of unique countries (if applicable)
            if 'country' in df.columns:
                num_countries = df['country'].nunique()
                print(f"Number of unique countries: {num_countries}")
            
            # Number of unique years (if applicable)
            if 'year' in df.columns:
                num_years = df['year'].nunique()
                print(f"Number of unique years: {num_years}")
            
            # Summary statistics for the dataset
            print(f"\nSummary statistics for {table_name} dataset:")
            print(df.describe())
            
            # Save the data to a CSV file
            output_path = os.path.join(output_directory, f"{table_name}.csv")
            df.to_csv(output_path, index=False)
            print(f"Data saved to {output_path}")
        else:
            print(f"Failed to fetch data from the {table_name} table.")
    except Exception as error:
        print(f"Error fetching data from {table_name}: {error}")

# List of tables to extract data from
tables = ['gdp', 'gdp_growth', 'gdp_percapita', 'gdp_ppp', 'gdp_ppp_percapita', 'gdp_unemployment', 'gdp_unemployment_by_category', 'inflation']

# Fetch data from each table and save to CSV
for table in tables:
    fetch_data_and_save_to_csv(table)


Data fetched successfully from gdp:
                                 id                      country code  year  \
0                        Aruba_1960                        Aruba  ABW  1960   
1  Africa Eastern and Southern_1960  Africa Eastern and Southern  AFE  1960   
2                  Afghanistan_1960                  Afghanistan  AFG  1960   
3   Africa Western and Central_1960   Africa Western and Central  AFW  1960   
4                       Angola_1960                       Angola  AGO  1960   

            gdp  
0           NaN  
1  1.931311e+10  
2  5.377778e+08  
3  1.040428e+10  
4           NaN  

High-level Assessment of gdp Data:
Number of records: 16226
Number of unique countries: 266
Number of unique years: 61

Summary statistics for gdp dataset:
               year           gdp
count  16226.000000  1.284000e+04
mean    1990.000000  1.125492e+12
std       17.607359  5.057450e+12
min     1960.000000  8.824448e+06
25%     1975.000000  2.366667e+09
50%     1990.000000